In [4]:
import os
from telethon.sync import TelegramClient
from telethon.tl.types import InputMessagesFilterPhotos
import csv

from datetime import datetime, timezone

In [5]:

# Configuration
api_id = '26559639'
api_hash = '3299ed6b2b0cbd2243470f18e1a77272'
phone = '+251984956023'  # Corrected: Phone number must be a string with quotes


# Load environment variables
#load_dotenv()

# Directory to save images
CSV_FILE = 'imageoflobeli.csv' 
SAVE_DIR = 'telegram_images'

In [6]:

# Create save directory if it doesn't exist
os.makedirs(SAVE_DIR, exist_ok=True)

# Connect to Telegram
client = TelegramClient(phone, api_id, api_hash)

# Function to download images
async def download_images(channel, start_date=None, end_date=None, max_images=None):
    image_count = 0
    downloaded_images = []  # List to hold details of downloaded images

    async for message in client.iter_messages(channel, filter=InputMessagesFilterPhotos):
        message_date = message.date.replace(tzinfo=timezone.utc)  # Make message date timezone aware
        if (start_date and message_date < start_date) or (end_date and message_date > end_date):
            continue
        if max_images and image_count >= max_images:
            break

        # Download the photo
        file_path = os.path.join(SAVE_DIR, f'{message.id}.jpg')
        await client.download_media(message.photo, file=file_path)
        
        # Append details to the list
        downloaded_images.append({
            'message_id': message.id,
            'file_path': file_path,
            'date': message_date.isoformat(),
            'caption': message.caption or ''
        })
        
        image_count += 1

    # Save details to CSV
    save_to_csv(downloaded_images)

def save_to_csv(data):
    with open(CSV_FILE, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['message_id', 'file_path', 'date', 'caption'])
        writer.writeheader()
        writer.writerows(data)

async def main():
    channel_name = 'lobelia4cosmetics'
    await download_images(
        channel_name,
        start_date=datetime(2024, 4, 1, tzinfo=timezone.utc),
        end_date=datetime(2024, 6, 10, tzinfo=timezone.utc),
        max_images=100
    )

# Start the client
async with client:
    await main()